In [1]:
import polars as pl
import polars_ds as pds

In [2]:
size = 100_000
df = pl.DataFrame({
    "a": ['a'] * 100 + ['b'] * 200 + ['c'] * 300,
    "b": [1] * 100 + [2] * 200 + [3] * 300
}).with_columns(
    pds.random_int(0,3).alias("c")
)
df.head()

a,b,c
str,i64,i32
"""a""",1,0
"""a""",1,2
"""a""",1,1
"""a""",1,0
"""a""",1,1


In [10]:
df.with_columns(
    pl.int_range(0, pl.len()).shuffle().over("a").alias("rank")
)

# .group_by("a").agg(
#     pl.col("a").alias("min"),
#     pl.col("a").alias("max"),
# )

a,min,max
str,i64,i64
"""a""",0,99
"""c""",0,299
"""b""",0,199


In [ ]:
(
    pl.concat(
    [
        df_sampled, 
        df_pl.select(pl.all().exclude(["a", "row_idx"]).take(df_sampled["row_idx"]))
    ], 
    how="horizontal")
)

In [ ]:
(pl.concat([df_sampled, df_pl.select(pl.all().exclude(["groups", "row_idx"])
                                     .take(df_sampled["row_idx"]))], how="horizontal"))

In [ ]:
from typing import Optional
def stratified_sample(df:pl.DataFrame, by:list[str], weights:Optional[str]) -> pl.DataFrame:


    pass

In [ ]:
df = pl.DataFrame({
    "a": [1,1,1,1,1,2,3,4,5],
    "b": [1,2,3,4,5,6,7,8,9],
})
df

In [ ]:
df.select(
    pl.col("b").rank(method="min")
)

In [ ]:
df = pl.DataFrame(
    {
        "id": [1, 2, 3],
        "val1": [0.1, 0.2, 5.0],
        "val2": [0.1, 0.3, 10.0],
        "val3": [0.1, 0.4, 11.0],
    }
)
df.head()

In [ ]:
x = [0.5, 0.5, 0.5]
dist = "cosine"
k = 1

test = df.filter(
    pds.query_knn_filter(pl.col("val1"), pl.col("val2"), pl.col("val3"), pt=x, dist=dist, k=k)
).select(pl.col("id"))
test